In [32]:
import pandas as pd
import missingno as msno 
import lightgbm as lgb

In [5]:
train=pd.read_csv(r'C:\Users\liuchang\Desktop\机器学习-王然老师\ML_dataset-作业数据集\final\train_final.csv')
test=pd.read_csv(r'C:\Users\liuchang\Desktop\机器学习-王然老师\ML_dataset-作业数据集\final\test_final.csv')

In [6]:
train.shape

(50000, 146)

In [7]:
test.shape

(50000, 146)

In [14]:
train.loan_status.value_counts()

1    39788
0    10212
Name: loan_status, dtype: int64

In [15]:
test.loan_status.value_counts()

1    40226
0     9774
Name: loan_status, dtype: int64

In [30]:
(train.isnull().sum()/len(train)).sort_values(ascending=False)

continuous_annual_inc_joint               0.99560
continuous_dti_joint                      0.99560
continuous_mths_since_last_record         0.81010
continuous_mths_since_last_major_derog    0.69896
continuous_mths_since_last_delinq         0.47834
                                           ...   
discrete_purpose_1_one_hot                0.00000
discrete_policy_code_1_one_hot            0.00000
discrete_home_ownership_4_one_hot         0.00000
discrete_home_ownership_3_one_hot         0.00000
continuous_annual_inc                     0.00000
Length: 146, dtype: float64

In [54]:
import io
import multiprocessing
from contextlib import redirect_stdout
from copy import deepcopy
from dataclasses import dataclass, asdict
import hyperopt.pyll
from hyperopt import fmin, tpe, hp
import numpy as np
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
# import torch

import copy
cpu_count = 4
use_gpu = False
@dataclass
class LGBOpt:
    num_threads: any = hp.choice('num_threads', [cpu_count])
    num_leaves: any = hp.choice('num_leaves', [64])
    metric: any = hp.choice('metric', ['binary_error'])
    num_round: any = hp.choice('num_rounds', [1000])
    objective: any = hp.choice('objective', ['binary'])
    learning_rate: any = hp.uniform('learning_rate', 0.01, 0.1)
    feature_fraction: any = hp.uniform('feature_fraction', 0.5, 1.0)
    bagging_fraction: any = hp.uniform('bagging_fraction', 0.8, 1.0)
    device_type: any = hp.choice('device_tpye', ['gpu']) if use_gpu else hp.choice('device_type',
                                                                                   ['cpu'])
    boosting: any = hp.choice('boosting', ['gbdt', 'dart', 'goss'])
    extra_trees: any = hp.choice('extra_tress', [False, True])
    drop_rate: any = hp.uniform('drop_rate', 0, 0.2)
    uniform_drop: any = hp.choice('uniform_drop', [True, False])
    lambda_l1: any = hp.uniform('lambda_l1', 0, 10)  # TODO: Check range
    lambda_l2: any = hp.uniform('lambda_l2', 0, 10)  # TODO: Check range
    min_gain_to_split: any = hp.uniform('min_gain_to_split', 0, 1)  # TODO: Check range
    min_data_in_bin = hp.choice('min_data_in_bin', [3, 5, 10, 15, 20, 50])

    @staticmethod
    def get_common_params():
        return {'num_thread': 4, 'num_leaves': 12, 'metric': 'binary', 'objective': 'binary',
                'num_round': 1000, 'learning_rate': 0.01, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
    


In [55]:
class FitterBase(object):
    def __init__(self, label, metric, max_eval=100, opt=None):
        self.label = label
        self.metric = metric
        self.opt_params = dict()
        self.max_eval = max_eval
        self.opt = opt

    def get_loss(self, y, y_pred):
        if self.metric == 'error':
            return 1 - accuracy_score(y, y_pred)
        elif self.metric == 'precision':
            return 1 - precision_score(y, y_pred)
        elif self.metric == 'recall':
            return 1 - recall_score(y, y_pred)
        elif self.metric == 'macro_f1':
            return 1 - f1_score(y, y_pred, average='macro')
        elif self.metric == 'micro_f1':
            return 1 - f1_score(y, y_pred, average='micro')
        elif self.metric == 'auc':  # TODO: Add a warning checking if y_predict is all [0, 1], it should be probability
            return 1 - roc_auc_score(y, y_pred)
        else:
            raise Exception("Not implemented yet.")

In [56]:
class LGBFitter(FitterBase):
    def __init__(self, label='label', metric='error', opt: LGBOpt = None, max_eval=100):
        super(LGBFitter, self).__init__(label, metric, max_eval)
        if opt is not None:
            self.opt = opt
        else:
            self.opt = LGBOpt()
        self.best_round = None
        self.clf = None

    def train(self, train_df, eval_df, params=None, use_best_eval=True):
        self.best_round = None
        dtrain = lgb.Dataset(train_df.drop(columns=[self.label]), train_df[self.label])
        deval = lgb.Dataset(eval_df.drop(columns=[self.label]), eval_df[self.label])
        evallist = [dtrain, deval]
        if params is None:
            use_params = deepcopy(self.opt_params)
        else:
            use_params = deepcopy(params)

        num_round = use_params.pop('num_round')
        if use_best_eval:
            with io.StringIO() as buf, redirect_stdout(buf):
                self.clf = lgb.train(use_params, dtrain, num_round, valid_sets=evallist)
                output = buf.getvalue().split("\n")
            min_error = np.inf
            min_index = 0
            for idx in range(len(output) - 1):
                if len(output[idx].split("\t")) == 3:
                    temp = float(output[idx].split("\t")[2].split(":")[1])
                    if min_error > temp:
                        min_error = temp
                        min_index = int(output[idx].split("\t")[0][1:-1])
            print("The minimum is attained in round %d" % (min_index + 1))
            self.best_round = min_index + 1
            return output
        else:
            with io.StringIO() as buf, redirect_stdout(buf):
                self.clf = lgb.train(use_params, dtrain, num_round, valid_sets=evallist)
                output = buf.getvalue().split("\n")
            self.best_round = num_round
            return output

    def search(self, train_df, eval_df, use_best_eval=True):
        self.opt_params = dict()

        def train_impl(params):
            self.train(train_df, eval_df, params, use_best_eval)
            if self.metric == 'auc':
                y_pred = self.clf.predict(eval_df.drop(columns=[self.label]), num_iteration=self.best_round)
            else:
                y_pred = (self.clf.predict(eval_df.drop(columns=[self.label]),
                                           num_iteration=self.best_round) > 0.5).astype(int)
            return self.get_loss(eval_df[self.label], y_pred)

        self.opt_params = fmin(train_impl, asdict(self.opt), algo=tpe.suggest, max_evals=self.max_eval)

    def search_k_fold(self, k_fold, data, use_best_eval=True):
        self.opt_params = dict()

        def train_impl_nfold(params):
            loss = list()
            for train_id, eval_id in k_fold.split(data):
                train_df = data.loc[train_id]
                eval_df = data.loc[eval_id]
                self.train(train_df, eval_df, params, use_best_eval)
                if self.metric == 'auc':
                    y_pred = self.clf.predict(eval_df.drop(columns=[self.label]), num_iteration=self.best_round)
                else:
                    y_pred = (self.clf.predict(eval_df.drop(columns=[self.label]),
                                               num_iteration=self.best_round) > 0.5).astype(int)
                loss.append(self.get_loss(eval_df[self.label], y_pred))
            return np.mean(loss)

        self.opt_params = fmin(train_impl_nfold, asdict(self.opt), algo=tpe.suggest, max_evals=self.max_eval)

    def train_k_fold(self, k_fold, train_data, test_data, params=None, drop_test_y=True, use_best_eval=True):
        acc_result = list()
        train_pred = np.empty(train_data.shape[0])
        test_pred = np.empty(test_data.shape[0])
        if drop_test_y:
            dtest = test_data.drop(columns=self.label)
        else:
            dtest = test_data

        models = list()
        for train_id, eval_id in k_fold.split(train_data):
            train_df = train_data.loc[train_id]
            eval_df = train_data.loc[eval_id]
            self.train(train_df, eval_df, params, use_best_eval)
            models.append(copy.deepcopy(self.clf))
            train_pred[eval_id] = self.clf.predict(eval_df.drop(columns=self.label), num_iteration=self.best_round)
            if self.metric == 'auc':
                y_pred = self.clf.predict(eval_df.drop(columns=[self.label]), num_iteration=self.best_round)
            else:
                y_pred = (self.clf.predict(eval_df.drop(columns=[self.label]),
                                           num_iteration=self.best_round) > 0.5).astype(int)
            acc_result.append(self.get_loss(eval_df[self.label], y_pred))
            test_pred += self.clf.predict(dtest, num_iteration=self.best_round)
        test_pred /= k_fold.n_splits
        return train_pred, test_pred, acc_result, models

In [57]:
fitter = LGBFitter(label='loan_status')

In [77]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5)
params = {'num_thread': 4, 'num_leaves': 12, 'metric': 'binary', 'objective': 'binary','num_round': 2000, 'learning_rate': 0.02, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
train_pred, test_pred, error_rate, models =fitter.train_k_fold(kfold, train, test, params = params)
print(params)
print(1-np.mean(error_rate))


The minimum is attained in round 421
Finished loading model, total used 2000 iterations
The minimum is attained in round 428
Finished loading model, total used 2000 iterations
The minimum is attained in round 399
Finished loading model, total used 2000 iterations
The minimum is attained in round 610
Finished loading model, total used 2000 iterations
The minimum is attained in round 297
Finished loading model, total used 2000 iterations
{'num_thread': 4, 'num_leaves': 12, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.02, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.91992


In [85]:
params = {'num_thread': 4, 'num_leaves': 16, 'metric': 'binary', 'objective': 'binary','num_round': 2000, 'learning_rate': 0.02, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
train_pred, test_pred, error_rate, models =fitter.train_k_fold(kfold, train, test, params = params)
print(params)
print(1-np.mean(error_rate))

The minimum is attained in round 394
Finished loading model, total used 2000 iterations
The minimum is attained in round 324
Finished loading model, total used 2000 iterations
The minimum is attained in round 345
Finished loading model, total used 2000 iterations
The minimum is attained in round 463
Finished loading model, total used 2000 iterations
The minimum is attained in round 285
Finished loading model, total used 2000 iterations
{'num_thread': 4, 'num_leaves': 16, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.02, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.91984


In [86]:
params = {'num_thread': 4, 'num_leaves': 16, 'metric': 'binary', 'objective': 'binary','num_round': 2000, 'learning_rate': 0.01, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
train_pred, test_pred, error_rate, models =fitter.train_k_fold(kfold, train, test, params = params)
print(params)
print(1-np.mean(error_rate))

The minimum is attained in round 859
Finished loading model, total used 2000 iterations
The minimum is attained in round 809
Finished loading model, total used 2000 iterations
The minimum is attained in round 748
Finished loading model, total used 2000 iterations
The minimum is attained in round 1062
Finished loading model, total used 2000 iterations
The minimum is attained in round 606
Finished loading model, total used 2000 iterations
{'num_thread': 4, 'num_leaves': 16, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.01, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.91964


In [87]:
params = {'num_thread': 4, 'num_leaves': 32, 'metric': 'binary', 'objective': 'binary','num_round': 2000, 'learning_rate': 0.01, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
train_pred, test_pred, error_rate, models =fitter.train_k_fold(kfold, train, test, params = params)
print(params)
print(1-np.mean(error_rate))

The minimum is attained in round 748
Finished loading model, total used 2000 iterations
The minimum is attained in round 558
Finished loading model, total used 2000 iterations
The minimum is attained in round 645
Finished loading model, total used 2000 iterations
The minimum is attained in round 518
Finished loading model, total used 2000 iterations
The minimum is attained in round 511
Finished loading model, total used 2000 iterations
{'num_thread': 4, 'num_leaves': 32, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.01, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.9193


In [88]:
params = {'num_thread': 4, 'num_leaves':10, 'metric': 'binary', 'objective': 'binary','num_round': 2000, 'learning_rate': 0.01, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
train_pred, test_pred, error_rate, models =fitter.train_k_fold(kfold, train, test, params = params)
print(params)
print(1-np.mean(error_rate))

The minimum is attained in round 993
Finished loading model, total used 2000 iterations
The minimum is attained in round 1036
Finished loading model, total used 2000 iterations
The minimum is attained in round 782
Finished loading model, total used 2000 iterations
The minimum is attained in round 1309
Finished loading model, total used 2000 iterations
The minimum is attained in round 676
Finished loading model, total used 2000 iterations
{'num_thread': 4, 'num_leaves': 10, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.01, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.91978


In [89]:
params = {'num_thread': 4, 'num_leaves':10, 'metric': 'binary', 'objective': 'binary','num_round': 2000, 'learning_rate': 0.015, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
train_pred, test_pred, error_rate, models =fitter.train_k_fold(kfold, train, test, params = params)
print(params)
print(1-np.mean(error_rate))

The minimum is attained in round 575
Finished loading model, total used 2000 iterations
The minimum is attained in round 786
Finished loading model, total used 2000 iterations
The minimum is attained in round 524
Finished loading model, total used 2000 iterations
The minimum is attained in round 853
Finished loading model, total used 2000 iterations
The minimum is attained in round 437
Finished loading model, total used 2000 iterations
{'num_thread': 4, 'num_leaves': 10, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.015, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.9198


In [97]:
params = {'num_thread': 4, 'num_leaves':10, 'metric': 'binary', 'objective': 'binary','num_round': 2000, 'learning_rate': 0.02, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
train_pred, test_pred, error_rate, models =fitter.train_k_fold(kfold, train, test, params = params)
print(params)
print(1-np.mean(error_rate))

The minimum is attained in round 423
Finished loading model, total used 2000 iterations
The minimum is attained in round 567
Finished loading model, total used 2000 iterations
The minimum is attained in round 457
Finished loading model, total used 2000 iterations
The minimum is attained in round 648
Finished loading model, total used 2000 iterations
The minimum is attained in round 383
Finished loading model, total used 2000 iterations
{'num_thread': 4, 'num_leaves': 10, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.02, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.91972


In [98]:
params = {'num_thread': 4, 'num_leaves':12, 'metric': 'binary', 'objective': 'binary','num_round': 2000, 'learning_rate': 0.02, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
train_pred, test_pred, error_rate, models =fitter.train_k_fold(kfold, train, test, params = params)
print(params)
print(1-np.mean(error_rate))

The minimum is attained in round 421
Finished loading model, total used 2000 iterations
The minimum is attained in round 428
Finished loading model, total used 2000 iterations
The minimum is attained in round 399
Finished loading model, total used 2000 iterations
The minimum is attained in round 610
Finished loading model, total used 2000 iterations
The minimum is attained in round 297
Finished loading model, total used 2000 iterations
{'num_thread': 4, 'num_leaves': 12, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.02, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.91992


In [99]:
params = {'num_thread': 4, 'num_leaves':16, 'metric': 'binary', 'objective': 'binary','num_round': 2000, 'learning_rate': 0.015, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
train_pred, test_pred, error_rate, models =fitter.train_k_fold(kfold, train, test, params = params)
print(params)
print(1-np.mean(error_rate))

The minimum is attained in round 546
Finished loading model, total used 2000 iterations
The minimum is attained in round 480
Finished loading model, total used 2000 iterations
The minimum is attained in round 649
Finished loading model, total used 2000 iterations
The minimum is attained in round 774
Finished loading model, total used 2000 iterations
The minimum is attained in round 403
Finished loading model, total used 2000 iterations
{'num_thread': 4, 'num_leaves': 16, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.015, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.9198


In [100]:
params = {'num_thread': 4, 'num_leaves':16, 'metric': 'binary', 'objective': 'binary','num_round': 2000, 'learning_rate': 0.025, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
train_pred, test_pred, error_rate, models =fitter.train_k_fold(kfold, train, test, params = params)
print(params)
print(1-np.mean(error_rate))

The minimum is attained in round 365
Finished loading model, total used 2000 iterations
The minimum is attained in round 303
Finished loading model, total used 2000 iterations
The minimum is attained in round 285
Finished loading model, total used 2000 iterations
The minimum is attained in round 244
Finished loading model, total used 2000 iterations
The minimum is attained in round 229
Finished loading model, total used 2000 iterations
{'num_thread': 4, 'num_leaves': 16, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.025, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.91966


In [101]:
params = {'num_thread': 4, 'num_leaves':20, 'metric': 'binary', 'objective': 'binary','num_round': 2000, 'learning_rate': 0.025, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
train_pred, test_pred, error_rate, models =fitter.train_k_fold(kfold, train, test, params = params)
print(params)
print(1-np.mean(error_rate))

The minimum is attained in round 286
Finished loading model, total used 2000 iterations
The minimum is attained in round 334
Finished loading model, total used 2000 iterations
The minimum is attained in round 264
Finished loading model, total used 2000 iterations
The minimum is attained in round 375
Finished loading model, total used 2000 iterations
The minimum is attained in round 210
Finished loading model, total used 2000 iterations
{'num_thread': 4, 'num_leaves': 20, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.025, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.91904


In [105]:
params = {'num_thread': 4, 'num_leaves':14, 'metric': 'binary', 'objective': 'binary','num_round': 2000, 'learning_rate': 0.01, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
train_pred, test_pred, error_rate, models =fitter.train_k_fold(kfold, train, test, params = params)
print(params)
print(1-np.mean(error_rate))

The minimum is attained in round 882
Finished loading model, total used 2000 iterations
The minimum is attained in round 773
Finished loading model, total used 2000 iterations
The minimum is attained in round 781
Finished loading model, total used 2000 iterations
The minimum is attained in round 1325
Finished loading model, total used 2000 iterations
The minimum is attained in round 600
Finished loading model, total used 2000 iterations
{'num_thread': 4, 'num_leaves': 14, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.01, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.91966


In [ ]:
for num_leaves in [10,11,12,13,14]:
    for learning_rate in [0.15,0.16,0.17,0.18,0.19,0.20,0.21,0.22,0.23,0.24,0.25]:
        params = {'num_thread': 4, 'num_leaves':num_leaves, 'metric': 'binary', 'objective': 'binary','num_round': 2000, 'learning_rate': learning_rate, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
        train_pred, test_pred, error_rate, models =fitter.train_k_fold(kfold, train, test, params = params)
        print(params)
        print(1-np.mean(error_rate))

{'num_thread': 4, 'num_leaves': 12, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.18, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.92014

In [109]:
params = {'num_thread': 4, 'num_leaves':12, 'metric': 'binary', 'objective': 'binary','num_round': 2000, 'learning_rate': 0.18, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
train_pred, test_pred, error_rate, models =fitter.train_k_fold(kfold, train, test, params = params)
print(params)
print(1-np.mean(error_rate))

The minimum is attained in round 48
Finished loading model, total used 2000 iterations
The minimum is attained in round 31
Finished loading model, total used 2000 iterations
The minimum is attained in round 36
Finished loading model, total used 2000 iterations
The minimum is attained in round 34
Finished loading model, total used 2000 iterations
The minimum is attained in round 34
Finished loading model, total used 2000 iterations
{'num_thread': 4, 'num_leaves': 12, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.18, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.92014


In [114]:
for num_leaves in [10,11,12,13,14]:
    for learning_rate in [0.015,0.016,0.017,0.018,0.019,0.020,0.021,0.022,0.023,0.024,0.025]:
        params = {'num_thread': 4, 'num_leaves':num_leaves, 'metric': 'binary', 'objective': 'binary','num_round': 2000, 'learning_rate': learning_rate, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
        train_pred, test_pred, error_rate, models =fitter.train_k_fold(kfold, train, test, params = params)
        print(params)
        print(1-np.mean(error_rate))

The minimum is attained in round 575
Finished loading model, total used 2000 iterations
The minimum is attained in round 786
Finished loading model, total used 2000 iterations
The minimum is attained in round 524
Finished loading model, total used 2000 iterations
The minimum is attained in round 853
Finished loading model, total used 2000 iterations
The minimum is attained in round 437
Finished loading model, total used 2000 iterations
{'num_thread': 4, 'num_leaves': 10, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.015, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.9198
The minimum is attained in round 661
Finished loading model, total used 2000 iterations
The minimum is attained in round 792
Finished loading model, total used 2000 iterations
The minimum is attained in round 551
Finished loading model, total used 2000 iterations
The minimum is attained in round 978
Finished loading model, total used 2000 iterations
The minimum is attained in ro

The minimum is attained in round 460
Finished loading model, total used 2000 iterations
The minimum is attained in round 852
Finished loading model, total used 2000 iterations
The minimum is attained in round 418
Finished loading model, total used 2000 iterations
{'num_thread': 4, 'num_leaves': 11, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.017, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.91984
The minimum is attained in round 484
Finished loading model, total used 2000 iterations
The minimum is attained in round 603
Finished loading model, total used 2000 iterations
The minimum is attained in round 429
Finished loading model, total used 2000 iterations
The minimum is attained in round 611
Finished loading model, total used 2000 iterations
The minimum is attained in round 370
Finished loading model, total used 2000 iterations
{'num_thread': 4, 'num_leaves': 11, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0

The minimum is attained in round 349
Finished loading model, total used 2000 iterations
{'num_thread': 4, 'num_leaves': 12, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.019, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.9199200000000001
The minimum is attained in round 421
Finished loading model, total used 2000 iterations
The minimum is attained in round 428
Finished loading model, total used 2000 iterations
The minimum is attained in round 399
Finished loading model, total used 2000 iterations
The minimum is attained in round 610
Finished loading model, total used 2000 iterations
The minimum is attained in round 297
Finished loading model, total used 2000 iterations
{'num_thread': 4, 'num_leaves': 12, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.02, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.91992
The minimum is attained in round 433
Finished loading model, total used 2000 iterations
The minimum is

The minimum is attained in round 347
Finished loading model, total used 2000 iterations
The minimum is attained in round 463
Finished loading model, total used 2000 iterations
The minimum is attained in round 317
Finished loading model, total used 2000 iterations
The minimum is attained in round 630
Finished loading model, total used 2000 iterations
The minimum is attained in round 281
Finished loading model, total used 2000 iterations
{'num_thread': 4, 'num_leaves': 13, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.022, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.9197
The minimum is attained in round 359
Finished loading model, total used 2000 iterations
The minimum is attained in round 323
Finished loading model, total used 2000 iterations
The minimum is attained in round 343
Finished loading model, total used 2000 iterations
The minimum is attained in round 457
Finished loading model, total used 2000 iterations
The minimum is attained in ro

The minimum is attained in round 301
Finished loading model, total used 2000 iterations
The minimum is attained in round 528
Finished loading model, total used 2000 iterations
The minimum is attained in round 237
Finished loading model, total used 2000 iterations
{'num_thread': 4, 'num_leaves': 14, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.024, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.91984
The minimum is attained in round 354
Finished loading model, total used 2000 iterations
The minimum is attained in round 335
Finished loading model, total used 2000 iterations
The minimum is attained in round 286
Finished loading model, total used 2000 iterations
The minimum is attained in round 370
Finished loading model, total used 2000 iterations
The minimum is attained in round 244
Finished loading model, total used 2000 iterations
{'num_thread': 4, 'num_leaves': 14, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0

## baseline
{'num_thread': 4, 'num_leaves': 10, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.019, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.92018